<a href="https://colab.research.google.com/github/KamronbekSolih/Violence_detection/blob/main/Copy_of_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!git clone https://github.com/talhaanwarch/pytorchvideo.git

Cloning into 'pytorchvideo'...
remote: Enumerating objects: 2353, done.
remote: Counting objects: 100% (72/72), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 2353 (delta 35), reused 59 (delta 30), pack-reused 2281
Receiving objects: 100% (2353/2353), 5.98 MiB | 12.33 MiB/s, done.
Resolving deltas: 100% (1307/1307), done.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install /content/pytorchvideo

Processing ./pytorchvideo
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 47.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pytorchvideo: filename=pytorchvideo-0.1.5-py3-none-any.whl size=201357 sha256=ed2d604e23e1d9ab61f1cae1a12bd7b9b063d16e85653bd2733a7b77d620a511
  Stored in directory: /tmp/pip-ephem-wheel-cache-a7soho8_/wheels/e9/3f/35/24b8f6ce8710265f8d4085a02b6d9a1311af57b1b54e876d58
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61400 sha256=90e1c27a9079449eabd88283ea824fea929df865d5e2866489965d6a477a29f7
  Stored in directory: /root/.cache/pip/wheels/01/c0/af/77c1cf53a1be9e42a52b48e5af2169d40ec2e89f7362489dd0
  Created wheel for iopath: filename=iopath-0.1.10-

In [5]:
from pytorchvideo.data import LabeledVideoDataset

In [6]:
from glob import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt



In [7]:
nvlc = glob("/content/drive/MyDrive/violence-dataset/Real Life Violence Dataset/NonViolence/*")
vlc = glob("/content/drive/MyDrive/violence-dataset/Real Life Violence Dataset/Violence/*")

label = [0]*len(nvlc)+[1]*len(vlc)

df = pd.DataFrame(zip(nvlc+vlc, label), columns = ['file', 'label'])
print('TV Content ', len(nvlc))
print('Advert ', len(vlc))
df.tail()




TV Content  1010
Advert  1000


,file,label
2005,/content/drive/MyDrive/violence-dataset/Real L...,1
2006,/content/drive/MyDrive/violence-dataset/Real L...,1
2007,/content/drive/MyDrive/violence-dataset/Real L...,1
2008,/content/drive/MyDrive/violence-dataset/Real L...,1
2009,/content/drive/MyDrive/violence-dataset/Real L...,1


In [45]:
df = pd.concat([df.head(600), df.tail(600)])
df

,file,label
0,/content/drive/MyDrive/violence-dataset/Real L...,0
1,/content/drive/MyDrive/violence-dataset/Real L...,0
2,/content/drive/MyDrive/violence-dataset/Real L...,0
3,/content/drive/MyDrive/violence-dataset/Real L...,0
4,/content/drive/MyDrive/violence-dataset/Real L...,0
...,...,...
2005,/content/drive/MyDrive/violence-dataset/Real L...,1
2006,/content/drive/MyDrive/violence-dataset/Real L...,1
2007,/content/drive/MyDrive/violence-dataset/Real L...,1
2008,/content/drive/MyDrive/violence-dataset/Real L...,1


In [46]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(df, test_size=0.2, shuffle = True)
len(train_df), len(val_df)

(640, 160)

In [47]:
train_df.head()

,file,label
70,/content/drive/MyDrive/violence-dataset/Real L...,0
1931,/content/drive/MyDrive/violence-dataset/Real L...,1
1943,/content/drive/MyDrive/violence-dataset/Real L...,1
1986,/content/drive/MyDrive/violence-dataset/Real L...,1
1945,/content/drive/MyDrive/violence-dataset/Real L...,1


Augmentation


In [48]:
from pytorchvideo.data import LabeledVideoDataset, make_clip_sampler, labeled_video_dataset

from pytorchvideo.transforms import (
    ApplyTransformToKey,
    Normalize,
    RandomShortSideScale,

    UniformTemporalSubsample,
    Permute
)

from  torchvision.transforms import (
    Compose,
    Lambda,
    RandomCrop,
    RandomHorizontalFlip,
    Resize

)

from torchvision.transforms._transforms_video import (
    CenterCropVideo,
    NormalizeVideo
)

In [49]:
video_transform = Compose([
    ApplyTransformToKey(key='video',
    transform=Compose([
        UniformTemporalSubsample(20),
        Lambda(lambda x:x/255),
        Normalize((0.45,0.45,0.45),(0.225,0.225,0.225)),
        RandomShortSideScale(min_size=248, max_size=256),
        CenterCropVideo(224),
        RandomHorizontalFlip(p=0.5)
    ])
    )
])

In [50]:
from torch.utils.data import DataLoader
train_dataset = labeled_video_dataset( train_df,
                                      clip_sampler=make_clip_sampler('random',2),
                                      transform=video_transform,
                                      decode_audio=False # for the smart tv project it will be true

                                      )

loader = DataLoader(train_dataset, batch_size= 5, num_workers = 0, pin_memory= True)


In [51]:
batch = next(iter(loader))

In [52]:
batch.keys()

dict_keys(['video', 'video_name', 'video_index', 'clip_index', 'aug_index', 'label'])

In [53]:
batch['video'].shape, batch['label'].shape

(torch.Size([5, 3, 20, 224, 224]), torch.Size([5, 1]))

Model Architecture

In [22]:
!pip install pytorch_lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.7/777.7 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 19.1 MB/s eta 0:00:00


In [23]:
import torch.nn as nn
import torch
from pytorch_lightning import LightningModule, seed_everything, Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor
from torch.optim.lr_scheduler import CosineAnnealingLR
from sklearn.metrics import classification_report
import torchmetrics


In [54]:
class OurModel(LightningModule):
    def __init__(self):
        super(OurModel, self).__init__()
        # model architecture
        self.video_model = torch.hub.load('facebookresearch/pytorchvideo', 'efficient_x3d_xs', pretrained=True)
        self.relu = nn.ReLU()
        self.linear = nn.Linear(400, 1)

        self.lr = 0.001
        self.batch_size = 4
        self.numworker = 2

        # evaluation metric
        self.metric = torchmetrics.Accuracy(task="binary")
        # loss function
        self.criterion = nn.BCEWithLogitsLoss()

    def forward(self, x):
        x = self.video_model(x)
        x = self.relu(x)
        x = self.linear(x)
        return x

    def configure_optimizers(self):
        opt = torch.optim.AdamW(params=self.parameters(), lr=self.lr)
        scheduler = CosineAnnealingLR(opt, T_max=10, eta_min=1e-6, last_epoch=-1)
        return {'optimizer': opt, 'lr_scheduler': scheduler}

    def train_dataloader(self):
        dataset = labeled_video_dataset(
            train_df,
            clip_sampler=make_clip_sampler('random', 2),
            transform=video_transform,
            decode_audio=False
        )

        loader = DataLoader(dataset, batch_size=self.batch_size, num_workers=self.numworker, pin_memory=True)
        return loader

    def training_step(self, batch, batch_idx):
        video, label = batch['video'], batch['label']
        out = self(video)
        loss = self.criterion(out, label)
        metric = self.metric(out, label.to(torch.int64))
        return {'loss': loss, 'metric': metric.detach()}

    def train_epoch_end(self, outputs):
        loss = torch.stack([x['loss'] for x in outputs]).mean().cpu().numpy().round(2)
        metric = torch.stack([x['metric'] for x in outputs]).mean().cpu().numpy().round(2)
        self.log('training_loss', loss)
        self.log('training_metric', metric)

    def val_dataloader(self):
        dataset = labeled_video_dataset(
            val_df,
            clip_sampler=make_clip_sampler('random', 2),
            transform=video_transform,
            decode_audio=False
        )

        loader = DataLoader(dataset, batch_size=self.batch_size, num_workers=self.numworker, pin_memory=True)
        return loader

    def validation_step(self, batch, batch_idx):
        video, label = batch['video'], batch['label']
        out = self(video)
        loss = self.criterion(out, label)
        metric = self.metric(out, label.to(torch.int64))
        return {'loss': loss, 'metric': metric.detach()}

    def on_validation_epoch_end(self):
        loss = torch.stack([x['loss'] for x in self.validation_epoch_output]).mean().cpu().numpy().round(2)
        metric = torch.stack([x['metric'] for x in self.validation_epoch_output]).mean().cpu().numpy().round(2)
        self.log('validation_loss', loss)
        self.log('validation_metric', metric)

    def test_dataloader(self):
        dataset = labeled_video_dataset(
            val_df,
            clip_sampler=make_clip_sampler('random', 2),
            transform=video_transform,
            decode_audio=False
        )

        loader = DataLoader(dataset, batch_size=self.batch_size, num_workers=self.numworker, pin_memory=True)
        return loader

    def test_step(self, batch, batch_idx):
        video, label = batch['video'], batch['label']
        out = self(video)
        return {'label': label.detach(), 'pred': out.detach()}

    def test_epoch_end(self, outputs):
        label = torch.cat([x['label'] for x in outputs]).cpu().numpy()
        pred = torch.cat([x['pred'] for x in outputs]).cpu().numpy()
        pred = np.where(pred > 0.5, 1, 0)
        print(classification_report(label, pred))


In [55]:
chekpoint_callback = ModelCheckpoint(
    monitor = 'val_loss',
    dirpath = 'chekpoints',
    filename = 'file',
    save_last = True
)
lr_monitor = LearningRateMonitor(logging_interval='epoch')

In [56]:
andoza = OurModel()

Using cache found in /root/.cache/torch/hub/facebookresearch_pytorchvideo_main


In [57]:
seed_everything(0)

INFO:lightning_fabric.utilities.seed:Seed set to 0


0

In [58]:
trainer = Trainer(max_epochs = 15,
                accelerator = 'gpu',
                devices = -1,
                precision = 16,
                # accumulate_grad_batches = 2,
                enable_progress_bar = False,
                num_sanity_val_steps = 0,
                callbacks = [lr_monitor, chekpoint_callback],
            #    limit_train_batches = 5,
             #   limit_val_batches = 1
                )

/usr/local/lib/python3.10/dist-packages/lightning_fabric/connector.py:558: `precision=16` is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [59]:
trainer.fit(andoza)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name        | Type              | Params
--------------------------------------------------
0 | video_model | EfficientX3d      | 3.8 M 
1 | relu        | ReLU              | 0     
2 | linear      | Linear            | 401   
3 | metric      | BinaryAccuracy    | 0     
4 | criterion   | BCEWithLogitsLoss | 0     
--------------------------------------------------
3.8 M     Trainable params
0         Non-trainable params
3.8 M     Total params
15.179    Total estimated model params size (MB)
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...
